In [1]:
#pip install geopy
import psycopg2
import pandas as pd
import geopy.distance
from geopy.geocoders import Nominatim

In [2]:


def distanca_od_centra_km(lokacija):
    geolocator = Nominatim(user_agent="MOJ_APP")
    #koordinate centra
    location = geolocator.geocode("Kneza Mihaila Beograd")
    coords_centar = (location.latitude, location.longitude)
    print(coords_centar)
    #koordinate lokacija
    location = geolocator.geocode(lokacija + " Beograd")
    if location != None:
        coords_lokacija = (location.latitude, location.longitude)
        print(coords_lokacija)
        distanca = geopy.distance.distance(coords_lokacija, coords_centar).km
        print(distanca)
        return distanca
    else:
        return None

In [3]:
#izdvojiti samo stanove koji se prodaju u Beogradu
x = []
y = []
try:
        conn = psycopg2.connect(database='nekretnine_database', user='postgres', password = '1234', host='localhost', port=5432)
        cursor = conn.cursor()
        
        stanovi_Beograd = "select *  from nekretnine_info where kategorija='stan' and grad='Beograd' and godina is not null and transakcija = 'Prodaja'  and lokacija is not null and spratnost is not null and kvadratura_m2 is not null and cena_eur is not null and broj_soba is not null;"
        cursor.execute(stanovi_Beograd)
        
        records = cursor.fetchall()
        col_names = []
        for elt in cursor.description:
            col_names.append(elt[0])
        df = pd.DataFrame(records, columns=col_names)
        print(df)
        
        df = df[['broj_soba', 'kvadratura_m2', 'spratnost', 'lokacija', 'cena_eur', 'godina']]
        df['spratnost'] = df['spratnost'].astype(int)
        print(df)
        
        #iskljuciti one vrednosti sa nekom null vrednoscu
        df = df.dropna()
        print(df)
        
        #odradili jednom i sacuvali u csv za svaki red primenili distanca_od_centra_km
        df['udaljenost'] = df.apply(lambda row : distanca_od_centra_km(row['lokacija']), axis = 1)
        print(df)
        
        #iskljuciti one vrednosti sa nekom null vrednoscu izracunate udaljenost
        df = df.dropna()
        print(df)
        
        x = df[['broj_soba', 'kvadratura_m2', 'spratnost', 'lokacija', 'godina', 'udaljenost']]
        print(x)
        
        y = df['cena_eur']
        print(y)
        
        
except (Exception, psycopg2.Error) as error:
        print("Neuspesno citanje iz baze", error)
finally:
        if conn:
            cursor.close()
            conn.close()

     kategorija transakcija            lokacija  godina  ukupan_broj_spratova  \
0          stan     Prodaja            Voždovac    2022                   5.0   
1          stan     Prodaja            Voždovac    2022                   5.0   
2          stan     Prodaja            Voždovac    2022                   5.0   
3          stan     Prodaja            Voždovac    2022                   5.0   
4          stan     Prodaja                Lion    2016                   5.0   
...         ...         ...                 ...     ...                   ...   
1320       stan     Prodaja        Hadžipopovac    1970                   NaN   
1321       stan     Prodaja            Voždovac    2008                   4.0   
1322       stan     Prodaja  Dedinje (RTV Pink)    1961                   4.0   
1323       stan     Prodaja            Voždovac    1979                  10.0   
1324       stan     Prodaja         Autokomanda    2017                   NaN   

                    grejanj

(44.8178925, 20.4569641)
(44.8020848, 20.3484997)
8.758272925844354
(44.8178925, 20.4569641)
(40.0590846, 22.5651069)
556.2923753901563
(44.8178925, 20.4569641)
(44.779101, 20.4130598)
5.5362987541185245
(44.8178925, 20.4569641)
(44.7916255, 20.4671457)
3.028100804547556
(44.8178925, 20.4569641)
(44.779101, 20.4130598)
5.5362987541185245
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.81227, 20.4817071)
2.054488014985931
(44.8178925, 20.4569641)
(44.79351665, 20.505533103949894)
4.701286657456922
(44.8178925, 20.4569641)
(44.8165217, 20.4184741)
3.0482574440179495
(44.8178925, 20.4569641)
(44.79791125, 20.486606838765624)
3.229503918362215
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.80005535, 20.475501390938568)
2.4656921514269445
(44.8178925, 20.4569641)
(44.8069189, 20.4178455)
3.326047466569787
(44.8178925, 20.4569641)
(44.9060854, 20.2747547)
17.41971612183071
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.8119025, 20.4242654)
2.67076

(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.8051519, 20.4776373)
2.1630935026910407
(44.8178925, 20.4569641)
(44.802097, 20.3643883)
7.5308193890106905
(44.8178925, 20.4569641)
(44.7986892, 20.5334006)
6.412357511311248
(44.8178925, 20.4569641)
(44.8013383, 20.487239454479354)
3.0199873882277415
(44.8178925, 20.4569641)
(44.7916255, 20.4671457)
3.028100804547556
(44.8178925, 20.4569641)
(44.80005535, 20.475501390938568)
2.4656921514269445
(44.8178925, 20.4569641)
(44.7981011, 20.4756422)
2.6496467107738804
(44.8178925, 20.4569641)
(44.7901689, 20.484559)
3.7759754581122
(44.8178925, 20.4569641)
(44.8119025, 20.4242654)
2.670760515983161
(44.8178925, 20.4569641)
(44.7981011, 20.4756422)
2.6496467107738804
(44.8178925, 20.4569641)
(44.8051519, 20.4776373)
2.1630935026910407
(44.8178925, 20.4569641)
(44.8136466, 20.5134098)
4.48966661360998
(44.8178925, 20.4569641)
(44.79791125, 20.486606838765624)
3.229503918362215
(44.8178925, 20.4569641)
(44.81877915, 20.467475246821568)
0.83

(44.8178925, 20.4569641)
(44.8592149, 20.3783579)
7.727643811926475
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.7613893, 20.4808961)
6.558459236390924
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.80005535, 20.475501390938568)
2.4656921514269445
(44.8178925, 20.4569641)
(44.80912255, 20.47062022382879)
1.4548943184742773
(44.8178925, 20.4569641)
(44.7779408, 20.4734682)
4.627809124287993
(44.8178925, 20.4569641)
(44.817362, 20.4715019)
1.151400209782773
(44.8178925, 20.4569641)
(44.8020808, 20.467175511162317)
1.9339117769294034
(44.8178925, 20.4569641)
(44.7779408, 20.4734682)
4.627809124287993
(44.8178925, 20.4569641)
(44.7841755, 20.5055572)
5.368480014073527
(44.8178925, 20.4569641)
(44.8020848, 20.3484997)
8.758272925844354
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.8445152, 20.4890986)
3.900039816858124
(44.8178925, 20.4569641)
(44.8203945, 20.5355783)
6.2241629532191105
(44.8178925, 20.4569641)
(44.8013383, 20.487239454479354)
3.0199873882277415
(44.

(44.8178925, 20.4569641)
(44.80005535, 20.475501390938568)
2.4656921514269445
(44.8178925, 20.4569641)
(44.79928525, 20.483573077955278)
2.950728658023147
(44.8178925, 20.4569641)
(44.74726335, 20.494950564764718)
8.404931935037915
(44.8178925, 20.4569641)
(44.8051519, 20.4776373)
2.1630935026910407
(44.8178925, 20.4569641)
(44.8203945, 20.5355783)
6.2241629532191105
(44.8178925, 20.4569641)
(44.8232255, 20.450872508168317)
0.7637809531403825
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.7748465, 20.5470888)
8.586996125221107
(44.8178925, 20.4569641)
(44.8014513, 20.3707829)
7.0581528069193915
(44.8178925, 20.4569641)
(44.7901689, 20.484559)
3.7759754581122
(44.8178925, 20.4569641)
(44.7901689, 20.484559)
3.7759754581122
(44.8178925, 20.4569641)
(44.7901689, 20.484559)
3.7759754581122
(44.8178925, 20.4569641)
(44.808437999999995, 20.456023863098622)
1.0532900558286078
(44.8178925, 20.4569641)
(44.8203945, 20.5355783)
6.2241629532191105
(44.8178925, 20.4569641)
(44.80912255, 20.

...

(44.8178925, 20.4569641)
(44.7997558, 20.5032678)
4.180902157560265
(44.8178925, 20.4569641)
(44.7779408, 20.4734682)
4.627809124287993
(44.8178925, 20.4569641)
(44.8745097, 20.4484695)
6.327538309738908
(44.8178925, 20.4569641)
(44.7997558, 20.5032678)
4.180902157560265
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.7594261, 20.4137684)
7.341600443143959
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.779101, 20.4130598)
5.5362987541185245
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.8104312, 20.4747404)
1.632392123413332
(44.8178925, 20.4569641)
(44.7897953, 20.4666487)
3.215018856368662
(44.8178925, 20.4569641)
(44.7521754, 20.4553526)
7.304095597826741
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.7913302, 20.4358707)
3.3908802817498596
(44.8178925, 20.4569641)
(44.7986892, 20.5334006)
6.412357511311248
(44.8178925, 20.4569641)
(44.7916255, 20.4671457)
3.028100804547556
(44.8178925, 20.4569641)
(44.7986892, 20.5334006)
6.412357511311248
(44.8178925, 20

(44.8178925, 20.4569641)
(44.7901689, 20.484559)
3.7759754581122
(44.8178925, 20.4569641)
(44.8136466, 20.5134098)
4.48966661360998
(44.8178925, 20.4569641)
(44.79791125, 20.486606838765624)
3.229503918362215
(44.8178925, 20.4569641)
(44.79791125, 20.486606838765624)
3.229503918362215
(44.8178925, 20.4569641)
(44.8051519, 20.4776373)
2.1630935026910407
(44.8178925, 20.4569641)
(44.81877915, 20.467475246821568)
0.8372037712279973
(44.8178925, 20.4569641)
(44.8196415, 20.4604306)
0.33608526663220517
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.79791125, 20.486606838765624)
3.229503918362215
(44.8178925, 20.4569641)
(44.7779408, 20.4734682)
4.627809124287993
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.8056809, 20.44890478397309)
1.4993440102972917
(44.8178925, 20.4569641)
(44.8051519, 20.4776373)
2.1630935026910407
(44.8178925, 20.4569641)
(44.7801523, 20.5229486)
6.696755717591663
(44.8178925, 20.4569641)
(44.7916255, 20.4671457)
3.028100804547556
(44.8178925, 20.45696

(44.7752952, 20.4987662)
5.774822769024017
(44.8178925, 20.4569641)
(44.8507665, 20.4052922)
5.480935161722186
(44.8178925, 20.4569641)
(44.779101, 20.4130598)
5.5362987541185245
(44.8178925, 20.4569641)
(44.7682225, 20.4923785)
6.19034668998448
(44.8178925, 20.4569641)
(44.8048741, 20.45427794148972)
1.46223157229707
(44.8178925, 20.4569641)
(44.9060854, 20.2747547)
17.41971612183071
(44.8178925, 20.4569641)
(44.80912255, 20.47062022382879)
1.4548943184742773
(44.8178925, 20.4569641)
(44.8028737, 20.4575104)
1.6695696906286124
(44.8178925, 20.4569641)
(44.8592149, 20.3783579)
7.727643811926475
(44.8178925, 20.4569641)
(44.8048741, 20.45427794148972)
1.46223157229707
(44.8178925, 20.4569641)
(44.7916255, 20.4671457)
3.028100804547556
(44.8178925, 20.4569641)
(44.7916255, 20.4671457)
3.028100804547556
(44.8178925, 20.4569641)
(44.81227, 20.4817071)
2.054488014985931
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.8196415, 20.4604306)
0.33608526663220517
(44.8178925, 20.4569641)
(4

(44.8056809, 20.44890478397309)
1.4993440102972917
(44.8178925, 20.4569641)
(44.7981011, 20.4756422)
2.6496467107738804
(44.8178925, 20.4569641)
(44.8439802, 20.4135954)
4.4906956347495095
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.7613893, 20.4808961)
6.558459236390924
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.8020848, 20.3484997)
8.758272925844354
(44.8178925, 20.4569641)
(44.81227, 20.4817071)
2.054488014985931
(44.8178925, 20.4569641)
(44.80912255, 20.47062022382879)
1.4548943184742773
(44.8178925, 20.4569641)
(44.7771209, 20.4925516)
5.334563424358191
(44.8178925, 20.4569641)
(44.8142441, 20.4577138)
0.40975379082458147
(44.8178925, 20.4569641)
(44.8196415, 20.4604306)
0.33608526663220517
(44.8178925, 20.4569641)
(44.8020848, 20.3484997)
8.758272925844354
(44.8178925, 20.4569641)
(44.8020848, 20.3484997)
8.758272925844354
(44.8178925, 20.4569641)
(44.779101, 20.4130598)
5.5362987541185245
(44.8178925, 20.4569

(44.7897953, 20.4666487)
3.215018856368662
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.3721181, 20.2746738)
51.60738518857496
(44.8178925, 20.4569641)
(44.8507665, 20.4052922)
5.480935161722186
(44.8178925, 20.4569641)
(44.8137802, 20.5139335)
4.529336276306716
(44.8178925, 20.4569641)
(44.817362, 20.4715019)
1.151400209782773
(44.8178925, 20.4569641)
(44.8119025, 20.4242654)
2.670760515983161
(44.8178925, 20.4569641)
(44.7913302, 20.4358707)
3.3908802817498596
(44.8178925, 20.4569641)
(44.826330049999996, 20.381233292641184)
6.0625307468511105
(44.8178925, 20.4569641)
(44.8204109, 20.422917528534363)
2.707399459121268
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.8178925, 20.4569641)
(44.7980504, 20.4955258)
3.764078274853733
(44.8178925, 20.4569641)
(44.79351665, 20.505533103949894)
4.701286657456922
(44.8178925, 20.4569641)
(44.7594261, 20.4137684)
7.341600443143959
(44.8178925, 20.4569641)
(44.7863098, 20.5082717)
5.366227568282904
(44.8178925, 20.4569641)
(44.799

In [4]:
print(x)
x = x.drop(columns = ['lokacija'])
print(x)
x.to_csv("with_distance.csv")

print(y)
y.to_csv("y_non_null.csv")


      broj_soba  kvadratura_m2  spratnost      lokacija  godina  udaljenost
0             3             74          3      Voždovac    2022    4.627809
1             3             63          2      Voždovac    2022    4.627809
2             2             53          3      Voždovac    2022    4.627809
3             5            120          1      Voždovac    2022    4.627809
4             4             85          4          Lion    2016    3.764078
...         ...            ...        ...           ...     ...         ...
1319          2             84          0     Konjarnik    2000    5.368480
1320          2             48          0  Hadžipopovac    1970    2.054488
1321          2             49          0      Voždovac    2008    4.627809
1323          1             44          0      Voždovac    1979    4.627809
1324          3             45         -1   Autokomanda    2017    3.215019

[1040 rows x 6 columns]
      broj_soba  kvadratura_m2  spratnost  godina  udaljenost
0